In [1]:
from urllib import parse

import os
import sys
sys.path.append(r"D:\往期\QHJ\ZERO")
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from pyquery import PyQuery as pq
import requests

from tools import *

In [2]:
start_time('get_goods')
phpsessid = "ik9uh45jlsaih896qopsupoa3s"

mgi = MallGoodsInfo(phpsessid)


In [3]:
doc = pq(mgi.get_fgoods())

获取首页商品信息


In [10]:
trs = doc('tbody tr')

In [34]:
trs[0].xpath('.//span[starts-with(@onclick,"opt_shelf")]')[0].text

'已上架'

In [3]:
def add_goods(doc):
    goods_l = []
    for tr in doc('tbody tr'):
        d = {}
        d['goods_id'] = tr.xpath('.//img')[0].attrib['goods_id']
        d['goods_name'] = tr.find_class('info_setting')[0].text.strip()
        d['send_price'] = tr.cssselect('input[key=send_price]')[0].attrib['value']
    #   d['send_price'] = tr.xpath('.//input[@key="send_price"]')[0].attrib['value']
    
        d['sort'] = tr.cssselect('input[key=sort]')[0].attrib['value']
    #   d['send_price'] = tr.xpath('.//input[@key="sort"]')[0].attrib['value']
    
        d['shelf'] = tr.cssselect('span[onclick^=opt_shelf]')[0].text
#         d['shelf'] = tr.xpath('.//span[starts-with(@onclick,"opt_shelf")]')[0].text
        
        
        d['tui'] = tr.cssselect('span[onclick^=opt_tui]')[0].text
#         d['tui'] = tr.xpath('.//span[starts-with(@onclick,"opt_tui")]')[0].text
        
        goods_l.append(d)
    return goods_l

In [4]:
goods_l = []
doc = pq(mgi.get_fgoods())
goods_l.extend(add_goods(doc))


In [5]:
# 获取其它页信息

item = doc('.pagination>li>a')
max_page = 0
l = []
def t(k,v):

    l.append( v.text)
item.each(t)
if l:
    max_page = int(l[-2])

for page in range(2 ,max_page+1):

    doc = pq(mgi.get_goods(page))
    goods_l.extend(add_goods(doc))


goods_data = pd.DataFrame(goods_l)

In [6]:
# 商品分类
goods_data['class_name'] = ""
goods_data['class_id'] = ""
for index,row in goods_data.iterrows():

    doc = pq(mgi.get_class(row['goods_id']))
    option = doc("select>option[selected]")[0]
    goods_data.loc[index,"class_id"] = option.attrib['value']
    goods_data.loc[index,"class_name"] = option.text


In [11]:
mgi.get_attrs(good['goods_id'])['select_list']

[{'attr_name_list': [{'attr_name': 'xx'}, {'attr_name': 'fasdasd'}],
  'attr_name': 'xxfasdasd',
  'attr_price': '123',
  'attr_stock': '123',
  'attt_unit': '',
  'attr_goods_code': 'yyy',
  'goods_id': '26',
  'attr_unit': 'sf',
  'attr_old_price': '300'},
 {'attr_name_list': [{'attr_name': 'tt'}],
  'attr_name': 'tt',
  'attr_price': '16',
  'attr_stock': '123',
  'attt_unit': '',
  'attr_goods_code': 'qq',
  'goods_id': '26',
  'attr_unit': 'g',
  'attr_old_price': '300'}]

In [7]:
# 获取属性信息
attr_l = []


for index,good in goods_data.iterrows():

    attr_l.extend(mgi.get_attrs(good['goods_id'])['select_list'])

attr_data = pd.DataFrame(attr_l)

In [8]:
def get_attrs(attrs):
    l = []
    for attr in attrs:

        l.append(attr['attr_name'])

    return "|".join(l)

attr_data['attr_name'] = attr_data["attr_name_list"].apply(get_attrs)
end_time('get_goods')

9.417999744415283

In [9]:
del attr_data['attt_unit']
del attr_data['attr_name_list']
del attr_data['attr_old_price']
goods_info = pd.merge(goods_data, attr_data,on='goods_id',how='left')

In [13]:
goods_info.columns

Index(['goods_id', 'goods_name', 'send_price', 'sort', 'class_name',
       'class_id', 'attr_goods_code', 'attr_name', 'attr_old_price',
       'attr_price', 'attr_stock', 'attr_unit'],
      dtype='object')

In [17]:

goods_info

,goods_id,goods_name,send_price,sort,class_name,class_id,attr_goods_code,attr_name,attr_price,attr_stock,attr_unit
0,1,法国大使（Delsey）双肩休闲包,0,10,背包,1,S0001P0001C0001P01,蓝,259,2000,个
1,1,法国大使（Delsey）双肩休闲包,0,10,背包,1,S0001P0001C0001P02,红,259,100,个
2,1,法国大使（Delsey）双肩休闲包,0,10,背包,1,S0001P0001C0001P03,灰,259,2000,个
3,2,法国大使（Delsey）商务电脑背包,0,100,背包,1,S0001P0001C0002P01,黑,215,100,个
4,2,法国大使（Delsey）商务电脑背包,0,100,背包,1,S0001P0001C0002P02,红,215,100,个
5,3,法国大使（Delsey）商务旅行背包,0,100,背包,1,S0001P0001C0003P01,黑,295,50,个
6,3,法国大使（Delsey）商务旅行背包,0,100,背包,1,S0001P0001C0003P02,灰,295,50,个
7,3,法国大使（Delsey）商务旅行背包,0,100,背包,1,S0001P0001C0003P03,蓝,295,50,个
8,6,LOVO经典床上四件套,0,100,床上用品,4,S0001P0002C0001P01,经典套装,0.1,50,套
9,7,iPhone 11 Pro MAX,0,100,智能手机,6,S0002P0001C0001P01,金色|64G,9300,100,部


In [14]:
print(goods_info)

   goods_id          goods_name send_price sort class_name class_id  \
0         1   法国大使（Delsey）双肩休闲包          0   10         背包        1   
1         1   法国大使（Delsey）双肩休闲包          0   10         背包        1   
2         1   法国大使（Delsey）双肩休闲包          0   10         背包        1   
3         2  法国大使（Delsey）商务电脑背包          0  100         背包        1   
4         2  法国大使（Delsey）商务电脑背包          0  100         背包        1   
5         3  法国大使（Delsey）商务旅行背包          0  100         背包        1   
6         3  法国大使（Delsey）商务旅行背包          0  100         背包        1   
7         3  法国大使（Delsey）商务旅行背包          0  100         背包        1   
8         6         LOVO经典床上四件套          0  100       床上用品        4   
9         7   iPhone 11 Pro MAX          0  100       智能手机        6   
10        7   iPhone 11 Pro MAX          0  100       智能手机        6   
11        7   iPhone 11 Pro MAX          0  100       智能手机        6   
12        7   iPhone 11 Pro MAX          0  100       智能手机        6   
13    